In [1]:
! pip show pika

In [1]:
import os
os.environ["TRANSFORMERS_CACHE"] = "/local/home/hfurquan/myProjects/Leaderboard/cache"
os.environ["HF_HOME"] = "/local/home/hfurquan/myProjects/Leaderboard/cache"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [2]:
import torch
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
from pathlib import Path

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize processor and model
processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
model = AutoModelForVision2Seq.from_pretrained(
    "ds4sd/SmolDocling-256M-preview",
    torch_dtype=torch.bfloat16
).to(DEVICE)
# ,_attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# Create input messages


/local/home/hfurquan/myProjects/govdocs-api/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/local/home/hfurquan/myProjects/govdocs-api/.venv/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [3]:
# Load images
image = load_image("example.jpg")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Convert this page to docling."}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=8192)
prompt_length = inputs.input_ids.shape[1]
trimmed_generated_ids = generated_ids[:, prompt_length:]
doctags = processor.batch_decode(
    trimmed_generated_ids,
    skip_special_tokens=False,
)[0].lstrip()

# Populate document
doctags_doc = DocTagsDocument.from_doctags_and_image_pairs([doctags], [image])
print(doctags)
# create a docling document
doc = DoclingDocument(name="Document")
doc.load_from_doctags(doctags_doc)

# export as any format
# HTML
output_path_html = Path("Out/") / "example.html"
doc.save_as_html(output_filoutput_path_htmle_path)
# MD
print(doc.export_to_markdown())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<doctag><page_header><loc_226><loc_16><loc_249><loc_22>- 7-</page_header>
<otsl><loc_12><loc_55><loc_476><loc_263><ecel><ched>1966<lcel><nl><ecel><ched>Tonnage  made(1)  tonnage  shipped<lcel><nl><fcel>Steel ingots (both carbon and alloy) (including  continuous cast steel):<ecel><ecel><nl><fcel>Basic open hearth ...................................... Electric .............................................. Oxygen ...................................................... Total steel ingots ...................................... Continuous cast steel included in above .......................... Alloy steel ingots included in above .......................... Steel castings (both carbon and alloy): Basic open hearth ...................................... Electric .............................................. Other ...................................................... Total steel castings ...................................... Alloy steel castings included in above: High alloy, except mangane

NameError: name 'output_filoutput_path_htmle_path' is not defined

In [4]:
output_path_html = Path("Out/") / "example.html"
doc.save_as_html(output_path_html)